In [16]:
import numpy as np
import pandas as pd
import requests, time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [17]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

In [19]:
songIds = []
base_url = 'https://www.melon.com/genre/song_list.htm?params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&params%5BdtlGnrCode%5D=params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&gnrCode=GN0'
genre_url = '#params%5BgnrCode%5D=GN0'
index_url = '&startIndex='
for i in tqdm(range(100, 801, 100)):
    main_url = f'{base_url}{i}{genre_url}{i}&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj'
    for k in range(1, 452, 50):
        page_url = f'{main_url}{index_url}{k}'
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('tbody > tr')
        for tr in trs:
            if tr.select_one('.bullet_icons.age_19'):
                continue
            songId = tr.select_one('.btn.button_icons.type03.song_info')['href'].split("'")[-2]
            songIds.append(songId)

100%|██████████| 8/8 [03:01<00:00, 22.73s/it]


In [4]:
len(songIds)

3970

In [30]:
infos = []
for songId in tqdm(songIds[:300]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1].replace(',', '')
        artist = info.select_one('.artist_name').get_text().replace(',', '')
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text().replace(',', '')
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text().strip().replace(',', '')
        composer = ul[1].select_one('.artist_name').get_text().strip().replace(',', '')
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '').replace(',', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').strip().replace(',', '')
        try:
            buttons = soup.select('.meta > button')
            plylstSeq = ' '.join([button['data-djcol-no'] for button in buttons])
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
        except:
            plylstSeq = np.nan
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue

  2%|▏         | 6/300 [00:16<14:19,  2.92s/it]

In [14]:
df = pd.DataFrame(infos)
df.shape[0]

255

In [15]:
df.to_csv('data/melon_genre11.csv', index=False)
pd.read_csv('data/melon_genre11.csv').head(2)


,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,32341095,막걸리 한잔,영탁,성인가요/트로트,내일은 미스터트롯 데스매치 PART1,류선우,류선우,20200130,https://cdnimg.melon.co.kr/cm2/album/images/10...,"1,335",49055,막걸리 한 잔\n온 동네 소문 났던 천덕꾸러기\n막내아들 장가가던 날\n앓던 이가 ...,425524406 457536753
1,34388834,참 좋은 날,이찬원,성인가요/트로트,참 좋은 날,이찬원,박종근,20211209,https://cdnimg.melon.co.kr/cm2/album/images/10...,394,16633,유난히 힘이 들던 날 꽃이 되어\n별이 되어 내게로 다가온 당신\n소리 내어 울고 ...,457536753 482886423
